# Prédiction

Import des différentes librairies

In [107]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn.impute import KNNImputer

from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import cross_validate

import pickle

Récupération de la base de données sous forme de dataframe :

In [108]:
df = pd.read_csv(r'C:\Users\Admin\Documents\marianneSimplon\simplon\immo_SiliconValley_marianneD\data\traindata_ori.csv',delimiter=',', decimal='.')

In [109]:
# Supression de la colonne Unnamed: 0
df = df.drop("Unnamed: 0", axis=1)

### Split du dataset en un dataset train et un dataset test

In [110]:
immoSV_train, immoSV_test = train_test_split(df, test_size=0.3)

## 1ère itération : données manquantes - suppression

In [111]:
immoSV_train.dropna(axis=0, inplace=True)
immoSV_test.dropna(axis=0, inplace=True)

<ipython-input-111-3fa0ee616d23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immoSV_train.dropna(axis=0, inplace=True)
<ipython-input-111-3fa0ee616d23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immoSV_test.dropna(axis=0, inplace=True)


## 4e itération : scaling par normalisation

In [112]:
immoSV_train_part = immoSV_train[["median_house_value","ocean_proximity"]]
immoSV_test_part = immoSV_test[["median_house_value","ocean_proximity"]]

In [113]:
ss_X_train = immoSV_train.drop(["median_house_value","ocean_proximity"], axis=1)
ss_X_test = immoSV_test.drop(["median_house_value","ocean_proximity"], axis=1)

##### Normalisation des données

In [114]:
sc = MinMaxScaler()
ft_sc = sc.fit(ss_X_train)
immoSV_train = ft_sc.transform(ss_X_train)
immoSV_train = pd.DataFrame(immoSV_train, columns=ss_X_train.columns)

In [115]:
immoSV_test = ft_sc.transform(ss_X_test)
immoSV_test = pd.DataFrame(immoSV_test, columns=ss_X_test.columns)

In [116]:
immoSV_train = immoSV_train.merge(immoSV_train_part,how='inner', on=immoSV_train.index)
immoSV_train = immoSV_train.drop('key_0', axis=1)
immoSV_test = immoSV_test.merge(immoSV_test_part,how='inner', on=immoSV_test.index)
immoSV_test = immoSV_test.drop('key_0', axis=1)

## 7e itération : encoding par OneHotEncoder

In [117]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = enc.fit(immoSV_train[['ocean_proximity']])
enc_df = pd.DataFrame(enc.transform(immoSV_train[['ocean_proximity']]).toarray())

In [118]:
immoSV_train = immoSV_train.merge(enc_df,how='inner', on=enc_df.index)
immoSV_train = immoSV_train.drop(['key_0','ocean_proximity'], axis=1)
immoSV_train.rename({0: 'NEAR_OCEAN', 1: 'NEAR_BAY',2: 'ISLAND', 3: 'INLAND', 4: 'H_OCEAN'}, inplace=True, axis=1)

In [119]:
enc_df = pd.DataFrame(enc.transform(immoSV_test[['ocean_proximity']]).toarray())
immoSV_test = immoSV_test.merge(enc_df,how='inner', on=enc_df.index)
immoSV_test = immoSV_test.drop(['key_0','ocean_proximity'], axis=1)
immoSV_test.rename({0: 'NEAR_OCEAN', 1: 'NEAR_BAY',2: 'ISLAND', 3: 'INLAND', 4: 'H_OCEAN'}, inplace=True, axis=1)

## 8e itération : feature permutation

In [120]:
immoSV_train = immoSV_train.drop(['H_OCEAN', 'ISLAND'],axis=1)
immoSV_test = immoSV_test.drop(['H_OCEAN', 'ISLAND'],axis=1)

## 10e itération : KNN Regression

# Model Pickle

In [121]:
immoSV_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,NEAR_OCEAN,NEAR_BAY,INLAND
0,0.606574,0.148778,0.784314,0.038134,0.063179,0.049190,0.063497,0.079316,94200.0,1.0,0.0,0.0
1,0.730080,0.015940,0.117647,0.153756,0.155387,0.106011,0.156111,0.271451,135000.0,0.0,0.0,0.0
2,0.618526,0.231668,0.274510,0.055204,0.055573,0.036901,0.048857,0.284486,118100.0,0.0,1.0,0.0
3,0.640438,0.113709,0.705882,0.031367,0.026389,0.013689,0.024840,0.680563,500001.0,1.0,0.0,0.0
4,0.284861,0.639745,1.000000,0.046631,0.060385,0.023247,0.057575,0.172646,103500.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11438,0.696215,0.066950,0.450980,0.051769,0.068147,0.028323,0.059878,0.125398,500001.0,0.0,0.0,0.0
11439,0.230080,0.605739,0.568627,0.052660,0.070320,0.035080,0.071558,0.103137,92900.0,0.0,1.0,0.0
11440,0.601594,0.149841,0.843137,0.039355,0.051226,0.038231,0.051489,0.241679,133500.0,1.0,0.0,0.0
11441,0.609562,0.149841,0.823529,0.042535,0.080255,0.076568,0.080770,0.119943,152800.0,1.0,0.0,0.0


In [122]:
y_train = immoSV_train["median_house_value"]
X_train = immoSV_train.drop(["median_house_value"], axis=1)
y_test = immoSV_test["median_house_value"]
X_test = immoSV_test.drop(["median_house_value"], axis=1)

In [123]:
X = pd.concat([X_train,X_test])
y = pd.concat([y_train,y_test])

In [124]:
my_model = pickle.load(open("model.pkl","rb"))
# my_model.predict(X, y)
# my_model.predict(X_test, y_test)
# my_model.score(X_test, y_test)

In [125]:
cv_results = cross_validate(my_model, X, y, cv=5)
# obtain the mean of scores
cv_results['test_score'].mean()

0.7060498109458216

In [129]:
predictions = my_model.predict(X)
predictions = pd.DataFrame(predictions)
predictions

,0
0,109160.0
1,207000.0
2,158100.0
3,436500.8
4,143920.0
...,...
16331,468640.6
16332,390240.0
16333,74420.0
16334,114760.0


In [ ]:
# pickle.dumps('model.pkl')